<a href="https://colab.research.google.com/github/jaroorhmodi/word2vec-and-BERT/blob/main/BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#BERT (Bidirectional Encoder Representations from Transformers)

In this notebook I will be replicating the model in the paper [**BERT: Pre-Training of Deep Bidirectional Transformers for Language Understanding**](https://arxiv.org/pdf/1810.04805.pdf).

While I will be creating the model from (mostly) scratch in PyTorch, I will not go into too much detail about why Multi-Head Attention is designed the way it is and how exactly the original [Transformer](https://arxiv.org/abs/1706.03762) architecture works. I have made another (*albeit messy*) [notebook that covers that paper](https://github.com/jaroorhmodi/transformer-from-scratch).

The model will be trained on the [**WikiText-2**](https://paperswithcode.com/dataset/wikitext-2) and [**Wikitext-103**](https://paperswithcode.com/dataset/wikitext-103) datasets.

In [ ]:
!pip install portalocker transformers tokenizers

In [ ]:
import os

import torch
from torch import nn
from torch.optim.lr_scheduler import LambdaLR
from torch.utils.data import DataLoader

import nltk
import numpy as np
import pandas as pd
import spacy

from tokenizers import BertWordPieceTokenizer
from transformers import AutoTokenizer

from torchtext.data import to_map_style_dataset
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torchtext.datasets import WikiText2, WikiText103 #our datasets for this project

from tqdm.auto import tqdm

DATASET_small = "WikiText2"
DATASET_large = "WikiText103"

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
TOKENIZER="basic_english"
DATA_DIRECTORY = "/content"

##Model Objective and Data

###Who (What) is BERT?


While BERT is a much more complex model and what it accomplishes isn't exactly akin to Word2Vec, the intuition behind both is similar. We pass in sentences and attempt to make a model learn how to represent text in a way that captures not only information about the tokens themselves but also something about their *meaning*.

Word2Vec does this by training a model on words and their context in sentences and learning  about their relationships with one another by either trying to predict context from words (*Skip-Gram*) or words from context (*CBOW*). The embeddings it produces are static for each word.

BERT trains a Transformer Encoder model on two specific objectives: *Masked Language Modeling* and *Next Sentence Prediction* to learn a wealth of information about tokens in their context and provide representations of them. Note that BERT is not simply learning static embeddings but rather representations that change based on context. Tokens in BERT are embedded using *WordPiece* embeddings.

The goal of the BERT paper was to introduce a way to represent words with a pre-trained transformer and not to make a model for a specific predictive goal. To this end it is trained in an unsupervised manner with the aforementioned MLM and NSP objectives (will be explained ahead).

###Data Processing

In [ ]:
#We need to pull in the dataset and break it into sentence pairs for the NSP objective
#and we need to mask random words and create objectives for the MLM objective.
DATASET = DATASET_small
dataset_class = WikiText2 if DATASET == DATASET_small else WikiText103
data_train = dataset_class(DATA_DIRECTORY, split = "train")
TOKENS_LOCATION = os.path.join(DATA_DIRECTORY, "datasets", DATASET, DATASET.lower()[:8]+f"-{DATASET[8:]}", "wiki.train.tokens")

MAX_TOKENIZED_SENTENCE_LEN = 128 #maximum number of tokens in sentence

gen = iter(data_train)

sample = []
for i in range(20):
  sample.append(next(gen))

In [ ]:
print(sample[:5])
print(len(sample))

[' \n', ' = Valkyria Chronicles III = \n', ' \n', ' Senjō no Valkyria 3 : <unk> Chronicles ( Japanese : 戦場のヴァルキュリア3 , lit . Valkyria of the Battlefield 3 ) , commonly referred to as Valkyria Chronicles III outside Japan , is a tactical role @-@ playing video game developed by Sega and Media.Vision for the PlayStation Portable . Released in January 2011 in Japan , it is the third game in the Valkyria series . <unk> the same fusion of tactical and real @-@ time gameplay as its predecessors , the story runs parallel to the first game and follows the " Nameless " , a penal military unit serving the nation of Gallia during the Second Europan War who perform secret black operations and are pitted against the Imperial unit " <unk> Raven " . \n', " The game began development in 2010 , carrying over a large portion of the work done on Valkyria Chronicles II . While it retained the standard features of the series , it also underwent multiple adjustments , such as making the game more <unk> for s

In the WikiText data, we see a lot of control characters like newlines, we want to make sure when we tokenize that we do not consider these. We also see that articles are delineated by a header given between single **=** signs and subheaders given by double, triple etc. equal signs.

We don't want header paragraphs and we don't want empty whitespace lines, so we need to preprocess the data some before we actually split them out into sentence pairs for our NSP task.

In [ ]:
tokenizer = BertWordPieceTokenizer(
    clean_text = True, #removes control chars like \n
    handle_chinese_chars = False, #not anticipating chinese chars
    strip_accents = False, #keep accents in
    lowercase = True #ignore case
)

tokenizer.train(
    files = TOKENS_LOCATION,
    vocab_size = 30_000,
    min_frequency = 10,
    limit_alphabet = 1000,
    wordpieces_prefix = '##',
    special_tokens=['[UNK]']
)

os.makedirs(os.path.join(DATA_DIRECTORY, "tokenizers"), exist_ok = True)

tokenizer.save_model(
    os.path.join(DATA_DIRECTORY, "tokenizers"),
    f"bert-wordpiece-{DATASET}"
)

['/content/tokenizers/bert-wordpiece-WikiText2-vocab.txt']

In [ ]:
test = sample[:4]
# test = "\n"
print(f"{test=}")
tokenized = tokenizer.encode_batch(test)
for i in range(len(tokenized)):
  example = tokenized[i]
  print(f"{example.tokens=}")
  print(f"{example.ids=}")
  print(f"{example.offsets=}")

test=[' \n', ' = Valkyria Chronicles III = \n', ' \n', ' Senjō no Valkyria 3 : <unk> Chronicles ( Japanese : 戦場のヴァルキュリア3 , lit . Valkyria of the Battlefield 3 ) , commonly referred to as Valkyria Chronicles III outside Japan , is a tactical role @-@ playing video game developed by Sega and Media.Vision for the PlayStation Portable . Released in January 2011 in Japan , it is the third game in the Valkyria series . <unk> the same fusion of tactical and real @-@ time gameplay as its predecessors , the story runs parallel to the first game and follows the " Nameless " , a penal military unit serving the nation of Gallia during the Second Europan War who perform secret black operations and are pitted against the Imperial unit " <unk> Raven " . \n']
example.tokens=[]
example.ids=[]
example.offsets=[]
example.tokens=['=', 'valkyria', 'chronicles', 'iii', '=']
example.ids=[29, 7750, 7829, 2903, 29]
example.offsets=[(1, 2), (3, 11), (12, 22), (23, 26), (27, 28)]
example.tokens=[]
example.ids=[]

Data preprocessing will require us to satisfy the two aforementioned training tasks.

All of the input text for BERT training is to be of the form

    "[CLS] <SENTENCE1> [SEP] <SENTENCE2> [SEP]"
with `[PAD]` tokens added at the end as needed.

In [ ]:
sample[0].strip() == ''

True

In [135]:
def new_article(para):
  #just checks if it is a new article
  return para.strip().startswith("= ") and not para.strip().startswith("= =")

def new_heading(para):
  #checks if it is a new heading
  #slightly different from the new article
  return para.strip().startswith("=")

def split_sentences(para):
  sentences = para.split('. ')

def article_sentence_pairs(article_sentences):
  #See note below
  sentence_pairs = []
  for i, sentence in enumerate(article_sentences):
    if i == len(article_sentences)-1:
      break
    else:
      sentence_pairs.append((sentence, article_sentences[i+1]))
  return sentence_pairs

def collect_sentence_pairs(dataset_iter):
  articles = []
  current_article = []
  for para in dataset_iter:
    if new_article(para):
      #new article, stop collecting sentences
      if len(current_article) > 0:
        articles += article_sentence_pairs(current_article)
      current_article = []
      continue
    if para.strip() == '' or new_heading(para):
      #new heading or empty line, skip and continue collecting
      continue
    current_article += para.strip().split('. ')
  return articles


NOTE: there are a few ways we could have split this dataset. I chose to go with one that splits at `". "` for this approach but we could have tokenized the data first and split according to maximum sequence length.

In [136]:
gen = iter(data_train)
sentence_pairs = collect_sentence_pairs(gen)
len(sps)

80096

In [138]:
sentence_pairs[77:80]

[("The rejected main theme was used as a hopeful tune that played during the game 's ending ",
  'The battle themes were designed around the concept of a " modern battle " divorced from a fantasy scenario by using modern musical instruments , constructed to create a sense of <unk> '),
 ('The battle themes were designed around the concept of a " modern battle " divorced from a fantasy scenario by using modern musical instruments , constructed to create a sense of <unk> ',
  'While Sakimoto was most used to working with synthesized music , he felt that he needed to incorporate live instruments such as orchestra and guitar '),
 ('While Sakimoto was most used to working with synthesized music , he felt that he needed to incorporate live instruments such as orchestra and guitar ',
  'The guitar was played by <unk> <unk> , who also arranged several of the later tracks ')]

So we see now that we are able to make pairs of sentences next to one another. Note that our particular method makes it so the first sentence from the immediately following paragraph is treated as a "next sentence" but the first sentence of the following article is not.

###The Two Training Objectives

  The following examples are straight from the paper and used to illustrate the NSP objective but can be used to explain the MLM objective as well.
    
    Input: [CLS] the man went to [MASK] store [SEP] he bought a gallon [MASK] milk [SEP]
    Label: IsNext

    Input: [CLS] the man went to [MASK] store [SEP] penguin [MASK] are flight ##less birds [SEP]
    Label: NotNext

####Masked Language Model
The **Masked Language Model** task masks out a given ratio of the tokens (and with some small probability substitutes with a random token) in the inputs and asks the model to predict the tokens that were masked out.

####Next Sentence Prediction

The **Next Sentence Prediction** task takes sentence pairs and creates a balanced classification task, where half of the time the following sentence is actually the next sentence and half of the time it is a random sentence. Then the model is trained to correctly predict whether or not the second sentence is the actual next sentence.

0

##Model Architecture

##Training